In [125]:
# %% Minimal setup
# If needed (uncomment in a notebook):
# !pip install requests python-dotenv

import os, json, textwrap, re, time
import requests
from typing import Tuple

# Environment/configuration (override via env vars)
API_KEY  = os.getenv("OPENAI_API_KEY", "cse476")
API_BASE = os.getenv("API_BASE", "http://10.4.58.53:41701/v1")
MODEL    = os.getenv("MODEL_NAME", "bens_model")

## API helper: `call_model_chat_completions`

This function calls an OpenAI-style `/v1/chat/completions` endpoint and returns a structured result. It is safe to run when the ASU API is reachable (on-campus or via VPN) and the `API_BASE`/`API_KEY` environment variables are set.

In [126]:
def call_model_chat_completions(prompt: str,
                                system: str = "You are a helpful assistant. Reply with only the final answer—no explanation.",
                                model: str = MODEL,
                                temperature: float = 0.0,
                                max_tokens: int = 128,
                                timeout: int = 60) -> dict:
    """
    Calls an OpenAI-style /v1/chat/completions endpoint and returns:
    { 'ok': bool, 'text': str or None, 'raw': dict or None, 'status': int, 'error': str or None, 'headers': dict }
    """
    url = f"{API_BASE}/chat/completions"
    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type":  "application/json",
    }
    payload = {
        "model": model,
        "messages": [
            {"role": "system", "content": system},
            {"role": "user",   "content": prompt}
        ],
        "temperature": temperature,
        "max_tokens": max_tokens,
    }

    try:
        resp = requests.post(url, headers=headers, json=payload, timeout=timeout)
        status = resp.status_code
        hdrs   = dict(resp.headers)
        if status == 200:
            data = resp.json()
            text = data.get("choices", [{}])[0].get("message", {}).get("content", "")
            return {"ok": True, "text": text, "raw": data, "status": status, "error": None, "headers": hdrs}
        else:
            # try best-effort to surface error text
            err_text = None
            try:
                err_text = resp.json()
            except Exception:
                err_text = resp.text
            return {"ok": False, "text": None, "raw": None, "status": status, "error": str(err_text), "headers": hdrs}
    except requests.RequestException as e:
        return {"ok": False, "text": None, "raw": None, "status": -1, "error": str(e), "headers": {}}

## Strategy 1 - Direct Answer (Single LLM Call)
- Gives the model a strict system prompt ensuring it only outputs the final answer
- Makes one call to the API
- Strips whitespace and returns result

In [127]:
def direct(question: str) -> Tuple[bool, str]:
    """
    Strategy 1: Directly ask the model the question.
    """
    directPrompt = "You are a helpful assistant, reply with only the final answer and give no explanations."
    systemResponse = call_model_chat_completions(question, system=directPrompt, temperature=0)
    
    if systemResponse.get('ok'):
        return True, systemResponse.get('text', '').strip()
    else:
        return False, systemResponse.get('error')

## Strategy 2 - Chain of Thought Loop

In [ ]:
def chain_of_thought(question: str) -> Tuple[bool, str]:
    """
    Strategy 2: Chain-of-Thought prompting.
    """
    cotPrompt = textwrap.dedent(f"""\
    You are a helpful assistant. Think through the problem step-by-step before providing the final answer.
    
    Question: {question}
    
    Let's think step by step.""")
    
    systemResponse = call_model_chat_completions(cotPrompt, temperature=0)
    
    if systemResponse.get('ok'):
        return True, systemResponse.get('text', '').strip()
    else:
        return False, systemResponse.get('error')

## Strategy 3 - Self Refinement

In [129]:
def self_refine(question: str, initial_answer: str) -> Tuple[bool, str]:
    """
    Strategy 3: Self-Refinement - The model reviews and refines its own answer.
    """
    systemPrompt = textwrap.dedent(
        "You are a meticulous assistant skilled in reviewing and refining answers. "\
        "Your task is to evaluate the provided answer to the question and improve it if necessary. "\
        "You will compare the initial answer against the question, identify any errors or omissions, "\
        "and provide a corrected and enhanced answer. "\
        "Follow these steps:\n"
        "1. Analyze the question and the initial answer carefully.\n"
        "2. Determine if the initial answer is correct and complete.\n"
        "3. If the answer is correct, repeat the correct answer. "\
        "Otherwise, provide only a corrected answer without any explanation."
    )
    selfRefinePrompt = textwrap.dedent(f"""\
        Question: {question}
        Initial Answer: {initial_answer}
        TASKS:
        Please provide a refined answer:
    """
    )
    systemResponse = call_model_chat_completions(selfRefinePrompt, system=systemPrompt, temperature=0)
    if systemResponse.get('ok'):
        return True, systemResponse.get('text', '').strip()
    else:
        return False, systemResponse.get('error')

## Agent Loop

In [130]:
def classify_question(question: str) -> Tuple[bool, str]:
    """
    Classify the question to determine the best strategy.
    """
    systemPrompt = "You are an expert question classifier. Analyze the question and determine the most suitable answering strategy: Direct, Chain-of-Thought, or Self-Refinement. Respond with only the strategy name."
    classifyPrompt = textwrap.dedent(f"""\
        Question: {question}
        TASK:
        Determine the best strategy to answer the question: Direct, Chain-of-Thought, or Self-Refinement.
        Respond with only the strategy name.
    """)
    classifyResponse = call_model_chat_completions(classifyPrompt, system=systemPrompt, temperature=0)
    if classifyResponse.get('ok'):
        return True, classifyResponse.get('text', '').strip()
    else:
        return False, classifyResponse.get('error')
    
def run_agent(question: str) -> Tuple[bool, str]:
    """
    Run the agent to answer the question using the classified strategy.
    """
    success, strategy = classify_question(question)
    if not success:
        return False, f"Classification Error: {strategy}"
    
    strategy = strategy.lower()
    if "direct" in strategy:
        return direct(question)
    elif "chain-of-thought" in strategy or "chain of thought" in strategy:
        return chain_of_thought(question)
    elif "self-refine" in strategy or "self refine" in strategy:
        # First get an initial answer
        success, initial_answer = direct(question)
        if not success:
            return False, f"Initial Answer Error: {initial_answer}"
        # Then refine it
        return self_refine(question, initial_answer)
    else:
        return False, f"Unknown Strategy: {strategy}"

In [131]:
# %% Demo agent
def demo():
    question = "Gunmen from Laredo starred which narrator of \"Frontier\"?"
    success, answer = run_agent(question)
    if success:
        print(f"Question: {question}\nAnswer: {answer}")
    else:
        print(f"Error: {answer}")

demo()

Question: Gunmen from Laredo starred which narrator of "Frontier"?
Answer: <thinking>
<steps>
<step>
<expert>Physicist</expert>
<take_away>Gunmen from Laredo are not a physical entity but a fictional group, so their association with a narrator of "Frontier" is not grounded in physical principles.</take_away>
<counterpoint>Physical principles do not apply to fictional narratives, so the question is not scientifically valid.</counterpoint>
<critique>As a physicist, I should have recognized that this is a question about media and not physics.</critique>
<likelihood>20%</likelihood>
</step>

<step>
<expert>Computer Scientist</expert>
<take_away>The question is not about algorithms or data structures, so it is not relevant to computer science.</take_away>
<counterpoint>Computer science is not applicable here, as the question is about media content.</counterpoint>
<critique>As a computer scientist, I should have recognized that this is a question about media and not computer science.</critiq